In [4]:
# Jupyter Notebook - 代码

# 导入必要的库
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import tensorflow_model_optimization as tfmot  # 新增剪枝库
import datetime  # 新增时间模块
# 设定日志级别
tf.get_logger().setLevel('ERROR')


# 🔹 超参数
IMG_SIZE = (128, 128)
AUTOTUNE = tf.data.AUTOTUNE

# -------------------------------- 第一阶段训练 ---------------------------------

In [5]:
# 🔹 数据集路径
base_dir = './dataset'
train_dir = os.path.join(base_dir, 'stage1/train')
valid_dir = os.path.join(base_dir, 'val')

BATCH_SIZE = 128
# 🔹 加载数据集
train_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, validation_split=0.001, subset="training", seed=12,
    batch_size=BATCH_SIZE, image_size=IMG_SIZE)

validation_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir, validation_split=0.999, subset="validation", seed=12,
    batch_size=BATCH_SIZE, image_size=IMG_SIZE)

class_names = train_dataset_raw.class_names
print("Class Names:", class_names)

# 预处理函数
def preprocess_image(image, label):
    return image, label

# 加载数据集
train_dataset = (train_dataset_raw
                 .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                 .cache()
                 .shuffle(1000)
                 .prefetch(AUTOTUNE))

validation_dataset = (validation_dataset_raw
                      .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                      .cache()
                      .prefetch(AUTOTUNE))


Found 2212 files belonging to 15 classes.
Using 2210 files for training.
Found 7878 files belonging to 15 classes.
Using 7870 files for validation.
Class Names: ['10_keyboard', '11_mobile_phone', '12_mouse', '13_headphones', '14_monitor', '15_speaker', '1_wrench', '2_soldering_iron', '3_electrodrill', '4_tape_measure', '5_screwdriver', '6_pliers', '7_oscillograph', '8_multimeter', '9_printer']


In [6]:
# 🔹 构建模型
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE, include_top=False, alpha=0.35, weights='imagenet')

# 冻结除最后4层外的所有层
base_model.trainable = True
# for layer in base_model.layers[:-0]:
#     layer.trainable = False

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])
model.build((None, 128, 128, 3))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 mobilenetv2_0.35_128 (Funct  (None, 4, 4, 1280)       410208    
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 15)                19215     
                                                                 
Total params: 429,423
Trainable params: 415,343
Non-trai

In [10]:
# 编译模型
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001, decay_steps=10000, decay_rate=0.90, staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# 训练第一阶段
early_stopping = tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)


In [12]:
history = model.fit(train_dataset, validation_data=validation_dataset,
                    epochs=20, callbacks=[early_stopping])

# 保存第一阶段模型
model.save('./model/stage1_model.h5')

Epoch 1/20
18/18 [==============================] - 4s 225ms/step - loss: 0.0153 - accuracy: 0.9991 - val_loss: 2.4282 - val_accuracy: 0.4855
Epoch 2/20
18/18 [==============================] - 4s 215ms/step - loss: 0.0115 - accuracy: 0.9995 - val_loss: 2.3941 - val_accuracy: 0.4900
Epoch 3/20
18/18 [==============================] - 4s 213ms/step - loss: 0.0116 - accuracy: 1.0000 - val_loss: 2.3481 - val_accuracy: 0.4950
Epoch 4/20
18/18 [==============================] - 4s 213ms/step - loss: 0.0121 - accuracy: 0.9991 - val_loss: 2.3298 - val_accuracy: 0.4968
Epoch 5/20
18/18 [==============================] - 4s 213ms/step - loss: 0.0106 - accuracy: 0.9995 - val_loss: 2.3197 - val_accuracy: 0.5000
Epoch 6/20
18/18 [==============================] - 4s 213ms/step - loss: 0.0089 - accuracy: 1.0000 - val_loss: 2.2615 - val_accuracy: 0.5079
Epoch 7/20
18/18 [==============================] - 4s 213ms/step - loss: 0.0085 - accuracy: 1.0000 - val_loss: 2.2116 - val_accuracy: 0.5158
Epoch 

# -------------------------------- 第二阶段训练 ---------------------------------

In [15]:
# 🔹 数据集路径
base_dir = './dataset'
train_dir = os.path.join(base_dir, 'stage2/train')
valid_dir = os.path.join(base_dir, 'val')

BATCH_SIZE = 128

# 加载数据集
train_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, validation_split=0.001, subset="training", seed=12,
    batch_size=BATCH_SIZE, image_size=IMG_SIZE)

validation_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir, validation_split=0.999, subset="validation", seed=12,
    batch_size=BATCH_SIZE, image_size=IMG_SIZE)

# 数据预处理（同第一阶段）
train_dataset = (train_dataset_raw
                 .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                 .cache()
                 .shuffle(1000)
                 .prefetch(AUTOTUNE))

validation_dataset = (validation_dataset_raw
                      .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                      .cache()
                      .prefetch(AUTOTUNE))

early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
# 加载第一阶段模型
model = tf.keras.models.load_model('./model/stage1_model.h5')

Found 8766 files belonging to 15 classes.
Using 8758 files for training.
Found 7878 files belonging to 15 classes.
Using 7870 files for validation.


In [16]:
history = model.fit(train_dataset, validation_data=validation_dataset,
                    epochs=100, callbacks=[early_stopping])

# 保存第一阶段模型
model.save('./model/stage2_model.h5')

Epoch 1/100
69/69 [==============================] - 23s 235ms/step - loss: 0.4482 - accuracy: 0.8592 - val_loss: 1.2432 - val_accuracy: 0.6309
Epoch 2/100
69/69 [==============================] - 9s 125ms/step - loss: 0.1373 - accuracy: 0.9581 - val_loss: 0.9603 - val_accuracy: 0.7122
Epoch 3/100
69/69 [==============================] - 9s 125ms/step - loss: 0.0755 - accuracy: 0.9794 - val_loss: 0.8784 - val_accuracy: 0.7388
Epoch 4/100
69/69 [==============================] - 9s 125ms/step - loss: 0.0459 - accuracy: 0.9895 - val_loss: 0.7614 - val_accuracy: 0.7801
Epoch 5/100
69/69 [==============================] - 9s 125ms/step - loss: 0.0316 - accuracy: 0.9938 - val_loss: 0.7035 - val_accuracy: 0.7999
Epoch 6/100
69/69 [==============================] - 9s 126ms/step - loss: 0.0215 - accuracy: 0.9969 - val_loss: 0.6892 - val_accuracy: 0.8061
Epoch 7/100
69/69 [==============================] - 9s 126ms/step - loss: 0.0166 - accuracy: 0.9978 - val_loss: 0.6665 - val_accuracy: 0.816

In [ ]:
# 🔹 数据集路径
base_dir = './dataset'
train_dir = os.path.join(base_dir, 'stage4/train')
valid_dir = os.path.join(base_dir, 'val')

BATCH_SIZE = 64

# 加载数据集
train_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, validation_split=0.001, subset="training", seed=12,
    batch_size=BATCH_SIZE, image_size=IMG_SIZE)

validation_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir, validation_split=0.999, subset="validation", seed=12,
    batch_size=BATCH_SIZE, image_size=IMG_SIZE)

# 数据预处理（同第一阶段）
train_dataset = (train_dataset_raw
                 .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                 .cache()
                 .shuffle(1000)
                 .prefetch(AUTOTUNE))

validation_dataset = (validation_dataset_raw
                      .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                      .cache()
                      .prefetch(AUTOTUNE))

# 加载第一阶段模型
model = tf.keras.models.load_model('./model/stage2_model.h5')

NotFoundError: Could not find directory ./dataset\stagee/train

In [18]:
history = model.fit(train_dataset, validation_data=validation_dataset,
                    epochs=100, callbacks=[early_stopping])

# 保存第一阶段模型
model.save('./model/stage2_model.h5')  

Epoch 1/100
274/274 [==============================] - 40s 103ms/step - loss: 0.1559 - accuracy: 0.9503 - val_loss: 0.6731 - val_accuracy: 0.8384
Epoch 2/100
274/274 [==============================] - 20s 71ms/step - loss: 0.0377 - accuracy: 0.9884 - val_loss: 0.5535 - val_accuracy: 0.8666
Epoch 3/100
274/274 [==============================] - 19s 71ms/step - loss: 0.0174 - accuracy: 0.9961 - val_loss: 0.6020 - val_accuracy: 0.8659
Epoch 4/100
274/274 [==============================] - 20s 73ms/step - loss: 0.0103 - accuracy: 0.9983 - val_loss: 0.5267 - val_accuracy: 0.8854
Epoch 5/100
274/274 [==============================] - 19s 70ms/step - loss: 0.0080 - accuracy: 0.9987 - val_loss: 0.6053 - val_accuracy: 0.8738
Epoch 6/100
274/274 [==============================] - 20s 74ms/step - loss: 0.0052 - accuracy: 0.9994 - val_loss: 0.4461 - val_accuracy: 0.9011
Epoch 7/100
274/274 [==============================] - 20s 72ms/step - loss: 0.0037 - accuracy: 0.9995 - val_loss: 0.4504 - val_a

# -------------------------------- 第三阶段训练 ---------------------------------

In [32]:
# 🔹 剪枝参数配置
PRUNING_PARAMS = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.30,
        final_sparsity=0.60,
        begin_step=0,
        end_step=2000,
        frequency=100
    )
}

# 加载第二阶段模型
model = tf.keras.models.load_model('./model/stage2_model.h5')

# 🔹 分离 Rescaling 层和基础模型
rescale_layer = model.layers[0]  # 提取 Rescaling 层
prunable_model = tf.keras.Sequential(model.layers[1:])  # 排除 Rescaling 后的模型

# 🔹 应用剪枝到可剪枝部分
with tfmot.sparsity.keras.prune_scope():  # 确保剪枝作用域正确
    pruned_model = tfmot.sparsity.keras.prune_low_magnitude(
        prunable_model, **PRUNING_PARAMS
    )

# 🔹 重新组合模型
final_model = tf.keras.Sequential([
    rescale_layer,  # 前置 Rescaling
    pruned_model    # 剪枝后的模型部分
])

# 编译模型
final_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [33]:
# 🔹 数据增强
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.125),
    tf.keras.layers.RandomZoom(0.5),
    tf.keras.layers.RandomTranslation(height_factor=0.25, width_factor=0.25),
    tf.keras.layers.RandomBrightness(0.25),
    tf.keras.layers.RandomContrast(0.5)
])

# 预处理函数（添加增强）
def preprocess_image_aug(image, label):
    image = data_augmentation(image)
    return image, label

In [ ]:
# 加载数据集（使用新数据集）
base_dir = './dataset'
train_dir = os.path.join(base_dir, 'stage4/train')
valid_dir = os.path.join(base_dir, 'val')

BATCH_SIZE = 64
train_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, validation_split=0.001, subset="training", seed=12,
    batch_size=BATCH_SIZE, image_size=IMG_SIZE)

validation_dataset_raw = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir, validation_split=0.999, subset="validation", seed=12,
    batch_size=BATCH_SIZE, image_size=IMG_SIZE)

# 数据预处理（应用增强）
train_dataset = (train_dataset_raw
                 .map(preprocess_image_aug, num_parallel_calls=AUTOTUNE)
                 .cache()
                 .shuffle(1000)
                 .prefetch(AUTOTUNE))

validation_dataset = (validation_dataset_raw
                      .map(preprocess_image, num_parallel_calls=AUTOTUNE)
                      .cache()
                      .prefetch(AUTOTUNE))

Found 27690 files belonging to 15 classes.
Using 22152 files for training.
Found 7878 files belonging to 15 classes.
Using 1575 files for validation.


In [35]:
# 🔹 添加剪枝回调
pruning_callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir='./logs_pruning')
]

In [ ]:
# 训练第三阶段
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=100,
    callbacks=[early_stopping, pruning_callbacks]
)


Epoch 1/100
347/347 [==============================] - 52s 74ms/step - loss: 0.0134 - accuracy: 0.9953 - val_loss: 0.5527 - val_accuracy: 0.8978
Epoch 2/100
347/347 [==============================] - 22s 63ms/step - loss: 0.0059 - accuracy: 0.9987 - val_loss: 0.4400 - val_accuracy: 0.9156
Epoch 3/100
347/347 [==============================] - 22s 63ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.4060 - val_accuracy: 0.9181
Epoch 4/100
347/347 [==============================] - 22s 63ms/step - loss: 7.2651e-04 - accuracy: 1.0000 - val_loss: 0.4017 - val_accuracy: 0.9225
Epoch 5/100
347/347 [==============================] - 22s 63ms/step - loss: 6.2278e-04 - accuracy: 1.0000 - val_loss: 0.4023 - val_accuracy: 0.9276
Epoch 6/100
347/347 [==============================] - 22s 64ms/step - loss: 4.1497e-04 - accuracy: 1.0000 - val_loss: 0.3584 - val_accuracy: 0.9314
Epoch 7/100
347/347 [==============================] - 23s 65ms/step - loss: 3.6961e-04 - accuracy: 1.0000 - val_loss:

In [ ]:
# 去除剪枝包装
final_model = tfmot.sparsity.keras.strip_pruning(model)
final_model.save('./model/stage3_pruned_final.h5')

In [ ]:
# 加载剪枝后的模型
model = tf.keras.models.load_model('./model/stage3_pruned_final.h5')

def representative_dataset():
    for image_batch, _ in tqdm(validation_dataset.take(500), desc="Processing"):
        yield [tf.cast(image_batch, tf.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model_quant = converter.convert()

# ---- 4. 动态生成带时间的文件名 ----
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_tflite_path = f'./model/model_{timestamp}.tflite'  # 新文件名格式

with open(output_tflite_path, 'wb') as f:
    f.write(tflite_model_quant)

target_dir = "model"
# 直接匹配当前目录下的 .h5 文件
for file in os.listdir(target_dir):
    if file.endswith(".h5"):
        file_path = os.path.join(target_dir, file)
        try:
            os.remove(file_path)
            print(f"已删除: {file_path}")
        except Exception as e:
            print(f"删除失败 [{file_path}]: {e}")

In [ ]:
# 混淆矩阵
y_pred = np.argmax(model.predict(validation_dataset), axis=1)
y_true = np.concatenate([labels.numpy() for _, labels in validation_dataset])

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()